In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
# CNN Model for classification
class classModel(nn.Module):
    def __init__(self):
        super(classModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 512 * 2 * 2)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [16]:
# Data transformation and loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [17]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [36]:
# Initialize and train the model
model = classModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train(model, loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(loader.dataset)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

train(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1, Loss: 1.4381
Epoch 2, Loss: 1.0271
Epoch 3, Loss: 0.8345
Epoch 4, Loss: 0.7097
Epoch 5, Loss: 0.5998
Epoch 6, Loss: 0.5068
Epoch 7, Loss: 0.4169
Epoch 8, Loss: 0.3566
Epoch 9, Loss: 0.2865
Epoch 10, Loss: 0.2417


In [34]:
# Evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [37]:
# Evaluate original model accuracy on clean test data
original_accuracy = evaluate_model(model, test_loader)
print(f'Original model accuracy on clean test data: {original_accuracy * 100:.2f}%')

Original model accuracy on clean test data: 81.11%


In [38]:
# FGSM attack function
def fgsm_attack(image, epsilon, data_grad):
    perturbed_image = image + epsilon * data_grad.sign()
    return torch.clamp(perturbed_image, 0, 1)

In [39]:
# Create adversarial examples and evaluate model
def evaluate_adversarial_examples(model, data_loader, epsilon):
    model.eval()
    correct = 0
    total = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        images.requires_grad = True
        outputs = model(images)
        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()
        data_grad = images.grad.data
        perturbed_data = fgsm_attack(images, epsilon, data_grad)
        
        outputs = model(perturbed_data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return correct / total

In [40]:
# Evaluate original model accuracy on adversarial test data
epsilon = 0.1
adversarial_accuracy = evaluate_adversarial_examples(model, test_loader, epsilon)
print(f'Original model accuracy on adversarial test data: {adversarial_accuracy * 100:.2f}%')

Original model accuracy on adversarial test data: 22.38%


In [54]:
# Implement adversarial training with learning rate scheduling
model.train()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()
num_epochs = 10  # Number of epochs for simplicity
alpha = 0.3  # Balance factor between clean and adversarial loss

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        images.requires_grad = True
        optimizer.zero_grad()
        
        # Forward pass with clean data
        outputs = model(images)
        clean_loss = criterion(outputs, labels)
        
        # Generate adversarial examples
        clean_loss.backward(retain_graph=True)
        data_grad = images.grad.data
        perturbed_data = fgsm_attack(images, epsilon, data_grad)
        
        # Zero gradients again
        optimizer.zero_grad()
        
        # Forward pass with perturbed data
        perturbed_outputs = model(perturbed_data)
        adversarial_loss = criterion(perturbed_outputs, labels)
        
        # Combine losses with balance factor alpha
        loss = alpha * clean_loss + (1 - alpha) * adversarial_loss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    # Step the learning rate scheduler
    scheduler.step()
    
    # Calculate average loss
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Evaluate model accuracy on clean test data
    clean_accuracy = evaluate_model(model, test_loader)
    
    # Evaluate model accuracy on adversarial test data
    adversarial_accuracy = evaluate_adversarial_examples(model, test_loader, epsilon)
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
          f"Clean Accuracy: {clean_accuracy * 100:.2f}%, "
          f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")

print("Adversarial training complete")


Epoch [1/10], Loss: 0.1547, Clean Accuracy: 77.03%, Adversarial Accuracy: 41.93%
Epoch [2/10], Loss: 0.0643, Clean Accuracy: 77.07%, Adversarial Accuracy: 45.74%
Epoch [3/10], Loss: 0.0463, Clean Accuracy: 76.71%, Adversarial Accuracy: 46.10%
Epoch [4/10], Loss: 0.0495, Clean Accuracy: 76.75%, Adversarial Accuracy: 44.85%
Epoch [5/10], Loss: 0.0478, Clean Accuracy: 77.01%, Adversarial Accuracy: 46.04%
Epoch [6/10], Loss: 0.0151, Clean Accuracy: 77.72%, Adversarial Accuracy: 46.66%
Epoch [7/10], Loss: 0.0041, Clean Accuracy: 77.71%, Adversarial Accuracy: 46.76%
Epoch [8/10], Loss: 0.0028, Clean Accuracy: 77.95%, Adversarial Accuracy: 47.10%
Epoch [9/10], Loss: 0.0023, Clean Accuracy: 77.91%, Adversarial Accuracy: 47.27%
Epoch [10/10], Loss: 0.0021, Clean Accuracy: 77.98%, Adversarial Accuracy: 47.61%
Adversarial training complete


In [55]:
# Evaluate adversarially trained model accuracy on clean test data
adv_trained_clean_accuracy = evaluate_model(model, test_loader)
print(f'Adversarially trained model accuracy on clean test data: {adv_trained_clean_accuracy * 100:.2f}%')

Adversarially trained model accuracy on clean test data: 77.98%


In [56]:
# Evaluate adversarially trained model accuracy on adversarial test data
adv_trained_adversarial_accuracy = evaluate_adversarial_examples(model, test_loader, epsilon)
print(f'Adversarially trained model accuracy on adversarial test data: {adv_trained_adversarial_accuracy * 100:.2f}%')

Adversarially trained model accuracy on adversarial test data: 47.61%


### Conclusions

#### Improved Robustness to Adversarial Examples:
The adversarially trained model shows a significant improvement in accuracy on adversarial test data, increasing from 22.38% to 47.61%. This indicates that the adversarial training process has successfully enhanced the model's robustness against adversarial attacks.

#### Slight Drop in Clean Accuracy:
There is a slight decrease in the model's accuracy on clean test data, dropping from 81.11% to 77.98%. This is a common trade-off in adversarial training, where improving robustness to adversarial examples can sometimes come at the cost of performance on clean data.

#### Balanced Trade-Off:
The reduction in clean accuracy is relatively small compared to the substantial gain in adversarial accuracy. This trade-off suggests that the model has achieved a better overall balance between robustness and accuracy.